In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from datetime import datetime
import re

%matplotlib inline

In [2]:
def parse_micro(file_name):
    poly_mods = [4096, 8192, 16384]
    coeff_lens = [1, 3, 8]
    
        
    re_time_seal_encode_avg = r'time_seal_encode_avg \(ns\) (.*)'
    re_time_he_encode_avg = r'time_he_encode_avg \(ns\) (.*)'
    
    re_time_seal_add_avg = r'time_seal_add_plain_avg \(ns\) (.*)'
    re_time_he_add_avg = r'time_he_add_plain_avg \(ns\) (.*)'
    
    re_time_seal_mult_avg = r'time_seal_multiply_plain_avg \(ns\) (.*)'
    re_time_he_mult_avg = r'time_he_multiply_plain_avg \(ns\) (.*)'
    
    re_poly_mod = r'.*poly_modulus_degree: (.*)'
 
    seal_encode_times = {}
    he_encode_times = {}
    seal_add_times = {}
    he_add_times = {}
    seal_mult_times = {}
    he_mult_times = {}

    
    with open(file_name) as f:
        lines = [line.strip() for line in f.readlines()]
        for line in lines:
            #print(line)
            if re.match(re_poly_mod, line):
                match = re.match(re_poly_mod, line)
                poly_mod = int(match.groups()[0])
            # Encode
            elif re.match(re_time_seal_encode_avg, line):
                match = re.match(re_time_seal_encode_avg, line)
                encode_time = float(match.groups()[0])
                seal_encode_times[poly_mod] = encode_time
            elif re.match(re_time_he_encode_avg, line):
                match = re.match(re_time_he_encode_avg, line)
                encode_time = float(match.groups()[0])
                he_encode_times[poly_mod] = encode_time
            
            # Add
            elif re.match(re_time_seal_add_avg, line):
                match = re.match(re_time_seal_add_avg, line)
                encode_time = float(match.groups()[0])
                seal_add_times[poly_mod] = encode_time
            elif re.match(re_time_he_add_avg, line):
                match = re.match(re_time_he_add_avg, line)
                encode_time = float(match.groups()[0])
                he_add_times[poly_mod] = encode_time
          
            # Mult
            elif re.match(re_time_seal_mult_avg, line):
                match = re.match(re_time_seal_mult_avg, line)
                encode_time = float(match.groups()[0])
                seal_mult_times[poly_mod] = encode_time
            elif re.match(re_time_he_mult_avg, line):
                match = re.match(re_time_he_mult_avg, line)
                encode_time = float(match.groups()[0])
                he_mult_times[poly_mod] = encode_time
                                          
            
    print('seal_encode_times', seal_encode_times)
    print('he_encode_times', he_encode_times)
    
    print('seal_add_times', seal_add_times)
    print('he_add_times', he_add_times)
    
    print('seal_mult_times', seal_mult_times)
    print('he_mult_times', he_mult_times)
    
           
    
    return (seal_encode_times, he_encode_times, seal_add_times, he_add_times, seal_mult_times, he_mult_times)

In [3]:
def make_add_table(seal_enc_times, he_enc_times):
    Ns = [4096, 8192, 16384]
    Ls = [1, 3, 8]
    
    # Runtime in ms
    header_str = r'''
\begin{tabular}{ c c c c c c }
\toprule
\multirow{2}{*}{$\boldsymbol N$} & \multirow{2}{*}{$\boldsymbol L$} & \multicolumn{2}{c}{\textbf{Runtime (ms)}} \\ \cmidrule{3-4}
& & \centering \textbf{General} & \centering \textbf{Scalar} & \centering{\textbf{Speedup}}
\tabularnewline
\midrule'''

    table_str = header_str
    for (N, L) in zip(Ns, Ls):
        # convert to micro seconds
        he_time = np.round(he_add_times[N] / 1000., 1)
        seal_time = np.round(seal_add_times[N] / 1000., 1)
        speedup = np.round(seal_time / float(he_time),1)
        table_str += '$2^{' + str(int(np.log2(N))) + '}$ & ' + str(L) + ' & ' + str(seal_time) + ' & ' \
                        + str(he_time) + ' & ' + str(speedup) + ' \\tabularnewline\n'
    
    print(table_str)

In [6]:
def make_mult_table(seal_enc_times, he_enc_times):
    Ns = [8192, 16384]
    Ls = [3, 8]
    
    # Runtime in ms
    header_str = r'''
\begin{tabular}{ c c c c c c }
\toprule
\multirow{2}{*}{$\boldsymbol N$} & \multirow{2}{*}{$\boldsymbol L$} & \multicolumn{2}{c}{\textbf{Runtime (ms)}} \\ \cmidrule{3-4}
& & \centering \textbf{General} & \centering \textbf{Scalar} & \centering{\textbf{Speedup}}
\tabularnewline
\midrule'''

    table_str = header_str
    for (N, L) in zip(Ns, Ls):
        # convert to micro seconds
        he_time = np.round(he_mult_times[N] / 1000., 1)
        seal_time = np.round(seal_mult_times[N] / 1000., 1)
        speedup = np.round(seal_time / float(he_time),1)
        table_str += '$2^{' + str(int(np.log2(N))) + '}$ & ' + str(L) + ' & ' + str(seal_time) + ' & ' \
                        + str(he_time) + ' & ' + str(speedup) + ' \\tabularnewline\n'
    
    
    print(table_str)

In [5]:
(seal_encode_times, he_encode_times, seal_add_times, he_add_times, seal_mult_times, he_mult_times) = parse_micro('./perf_micro.txt')
make_add_table(seal_add_times, he_add_times)

seal_encode_times {4096: 570.0, 8192: 3004.0, 16384: 27880.0}
he_encode_times {4096: 367.0, 8192: 404.0, 16384: 706.0}
seal_add_times {4096: 2356.0, 8192: 12735.0, 16384: 120804.0}
he_add_times {4096: 949.0, 8192: 4553.0, 16384: 29230.0}
seal_mult_times {4096: 0.0, 8192: 182599.0, 16384: 983490.0}
he_mult_times {4096: 0.0, 8192: 71900.0, 16384: 386740.0}

\begin{tabular}{ c c c c c c }
\toprule
\multirow{2}{*}{$\boldsymbol N$} & \multirow{2}{*}{$\boldsymbol L$} & \multicolumn{2}{c}{\textbf{Runtime (ms)}} \\ \cmidrule{3-4}
& & \centering \textbf{General} & \centering \textbf{Scalar} & \centering{\textbf{Speedup}}
\tabularnewline
\midrule$2^{12}$ & 1 & 2.4 & 0.9 & 2.7 \tabularnewline
$2^{13}$ & 3 & 12.7 & 4.6 & 2.8 \tabularnewline
$2^{14}$ & 8 & 120.8 & 29.2 & 4.1 \tabularnewline



In [7]:
make_mult_table(seal_add_times, he_add_times)


\begin{tabular}{ c c c c c c }
\toprule
\multirow{2}{*}{$\boldsymbol N$} & \multirow{2}{*}{$\boldsymbol L$} & \multicolumn{2}{c}{\textbf{Runtime (ms)}} \\ \cmidrule{3-4}
& & \centering \textbf{General} & \centering \textbf{Scalar} & \centering{\textbf{Speedup}}
\tabularnewline
\midrule$2^{13}$ & 3 & 182.6 & 71.9 & 2.5 \tabularnewline
$2^{14}$ & 8 & 983.5 & 386.7 & 2.5 \tabularnewline

